In [ ]:
#!/usr/bin/env python
# coding: utf-8
#license = Apache 2.0 open source code
#Extracts date, lat, lon from Saildrone data 
#Code adapted from Chelle Gentemann, Johannes Karstensen, Senya Stein, Tobias Koelling 

#import relevant libraries and functions
import numpy as np
from ipyleaflet import GeoJSON
import requests
import time
import datetime
import dateutil.parser as dparser


def get_key(file_name):
    myvars = {}
    with open(file_name) as myfile:
        for line in myfile:
            name, var = line.partition("=")[::2]
            myvars[name.strip()] = str(var).rstrip()
    return myvars

file_key = "C:/Users/gentemann/Google Drive/f_drive/secret_keys/saildrone_eurec4a_v2.txt"
saildrone_key = get_key(file_key)

In [ ]:
#converts time to Unix timestamp

def convertunix(date):
    
    #we don't need the Z at the end
    date = np.datetime64(date.replace('Z', ''))
    timestamp = (date - np.datetime64('1970-01-01T00:00:00')) / np.timedelta64(1, 's')
    return timestamp


#extracts latitude
def coordlat(coords):
    coords = dict_assets['features'][0]['geometry']['coordinates']
    lat = coords[0]
    return lat
    
#extracts longitude
def coordlon(coords):
    coords = dict_assets['features'][0]['geometry']['coordinates']
    lon = coords[1]
    return lon   


def put_asset_info_dict_geomar(dict_assets):
    asset_dictionary = {} #this will store the final lat lon and time
    i,isave = 0,0

    #finds number of assets to extract
    number_of_assets = len(dict_assets['features'])
    print('Asset Number = ' + str(number_of_assets))

    #adds each asset to dictionary to output
    while i < number_of_assets:

        timestamp = convertunix(dict_assets['features'][i]['properties']['obs_timestamp']) #timestamp
        aplatform_id = dict_assets['features'][i]['properties']['platform_id'] #platform id
        atype = dict_assets['features'][i]['properties']['type'] #platform type
        aname = dict_assets['features'][i]['properties']['name'] #platform type
        rlat = coordlon(dict_assets['features'][i]['geometry']['coordinates']) #latitude
        rlon = coordlon(dict_assets['features'][i]['geometry']['coordinates']) #longitude
        rsog = dict_assets['features'][i]['properties']['speed_over_ground'] 
        rheading = dict_assets['features'][i]['properties']['heading'] 
        i+=1

        if atype=='Saildrone':
            continue

        if rsog=='None':
            rsog=Np.NaN
        if rheading=='None':
            rheading=np.NaN

        #final dictionary of all relevant information (frome above)
        asset_dictionary[isave] = { 
          "type" : atype,
          "name" : aname,
          "mmsi" : aplatform_id,
          "latitude" : rlat,
          "longitude" : rlon,
          "timestamp" : np.int(timestamp) ,
          "hdg": rheading,
          "sog": rsog #,
         # "token": saildrone_key['token']
          }
        #print(asset_dictionary[i])
        isave+=1
    return asset_dictionary


In [ ]:
#get SWIFT buoy data
BUOYS = [
    "SWIFT 16",
    "SWIFT 17",
    "SWIFT 22",
    "SWIFT 23",
    "SWIFT 24",
    "SWIFT 25",
]

def parse_buoy_data(d):
    d = d.copy()
#    d["time"] = dparser.parse(d["timestamp"]).replace(tzinfo=None)
    d["time"] = convertunix(d["timestamp"])
#    print(d["time"],d["timestamp"])
#    del d["timestamp"]
    del d["wind_speed"]
    del d["wave_height"]
    del d["voltage"]
    for k, v in list(d.items()):
        if v is None:
            del d[k]
    return d

def get_swift_buoy(name, max_age=datetime.timedelta(days=10)):
    now = datetime.datetime.utcnow()
    start = now - max_age
    params = {
        "action": "kml",
        "buoy_name": name,
        "start": start.isoformat(),
        "end": "",
        "format": "json",
    }

    res = requests.get("http://swiftserver.apl.washington.edu/kml", params=params)
    res = res.json()
    if not res.get("success", False):
        raise RuntimeError("unsuccessfull response")
    buoys = [b for b in res["buoys"] if b["name"] == name]
    if len(buoys) != 1:
        raise ValueError("could not uniquely identify buoy \"{}\", {} results".format(name, len(buoys)))
    data = list(sorted(map(parse_buoy_data, buoys[0]["data"]),
                       key=lambda x: x["time"]))

    return data

def put_asset_info_dict_swift(buoy,dict_assets):
    asset_dictionary = {} #this will store the final lat lon and time
    i = 0

    #finds number of assets to extract
    number_of_assets = len(dict_assets)
    print('Asset Number = ' + str(number_of_assets))

    #adds each asset to dictionary to output
    while i < number_of_assets:

        timestamp = convertunix(dict_assets[i]["timestamp"]) #timestamp
        aplatform_id = 'SW'+buoy[-2:] #dict_assets['features'][i]['properties']['platform_id'] #platform id
        atype = 'swift' #platform type
        aname = buoy #platform type
        rlat = dict_assets[i]["lat"] #latitude
        rlon = dict_assets[i]["lon"] #longitude

        #final dictionary of all relevant information (frome above)
        asset_dictionary[i] = { 
          "type" : atype,
          "name" : aname,
          "mmsi" : aplatform_id,
          "latitude" : rlat,
          "longitude" : rlon,
          "timestamp" : np.int(timestamp) 
         # "token": saildrone_key['token']
          }
        #print(asset_dictionary[i])
        i+=1
        
    return asset_dictionary


In [ ]:
#web requests using requests
wfs_base_url = "https://maps.geomar.de/geoserver/OceanEddies/wfs" #server location
requested_layer = "OceanEddies:current_positions"
#dictionary of data
requests_parameters = {
                        'service': 'wfs',
                        'version': '1.0.0',
                        'request': 'getFeature',
                        'typename': requested_layer,
                        #filter the results on the server
                       # 'CQL_FILTER': "type like 'Saildrone'",  # OCG CQL filter
                        'viewparams': "maxage:48",  # parameters for custom view on geoserver: set max age of position to 48h 
                        "outputformat": "application/json"
                      }
response = requests.get(wfs_base_url, params=requests_parameters)
#print(f'use the following link for your query: {response.url}')

wg_positions = GeoJSON(data=response.json(), hover_style={'fillColor': 'red'}, name='WaveGlider Positions') 
#print(response.json())


#assign the web response to dict_assets (a dictionary containing all the assets)

dict_assets = response.json()
dict_assets

#assign dict values to variables 
dict_assets['features'][0]['properties']['obs_timestamp']
dict_assets['features'][0]['properties']['type']
dict_assets['features'][0]['geometry']['coordinates']
date = dict_assets['features'][0]['properties']['obs_timestamp']                          

asset_dictionary_geomar = put_asset_info_dict_geomar(dict_assets)

print('saved assets',len(asset_dictionary_geomar))
for i in asset_dictionary_geomar:
    print(asset_dictionary_geomar[i]['name'])

In [ ]:
for buoy in BUOYS:
    try:
        data = get_swift_buoy(buoy)
        latest = data[-1]
        asset_dictionary_swift = put_asset_info_dict_swift(buoy,data)
    except ValueError:
        continue
print(len(asset_dictionary_swift))

In [ ]:
#merge the two dictionaries
print(len(asset_dictionary_geomar))
print(len(asset_dictionary_swift))
asset_dictionary_all=asset_dictionary_geomar
ilen = len(asset_dictionary_all)
for i in asset_dictionary_swift:
    asset_dictionary_all[i+ilen]=asset_dictionary_swift[i]
    

print(len(asset_dictionary_all))

In [ ]:
#post to saildrone api
#do NOT commit any changes with headers printed out because it has token
for iasset in asset_dictionary_all:
    payload=asset_dictionary_all[iasset] #{'key': saildrone_key['key'], 'secret':saildrone_key['secret']}
    headers={'Content-Type':'application/json', 'Accept':'application/json','authorization':saildrone_key['token']}
    url = 'https://developer-mission.saildrone.com/v1/ais'
    res = requests.post(url, json=payload, headers=headers)


In [ ]:
#payload